In [20]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda' if USE_CUDA else 'cpu')
print(DEVICE)
BATCH_SIZE = 224

cuda


In [21]:
class Net(nn.Module):
    def __init__(self):
        
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(3, 32, 3, padding = 1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1)
        self.conv3 = nn.Conv2d(64, 64, 3, padding = 1)
        
        self.fc1 = nn.Linear(4096, 512)
        self.fc2 = nn.Linear(512, 450)
        
    def forward(self, x):
        
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25, training=self.training)
        
        x = x.view(-1, 4096)
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        
        return F.log_softmax(x, dim=1)

In [22]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        test_accuracy = 100. * correct / len(test_loader.dataset)
        return test_loss, test_accuracy

In [23]:
transform_base = transforms.Compose([transforms.Resize([64, 64]), transforms.ToTensor()])
test_dataset = ImageFolder(root='./dataset/test', transform=transform_base)
test_loader_base = torch.utils.data.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

In [24]:
transform_resNet = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.RandomCrop(52),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_resNet = ImageFolder(root='./dataset/test', transform=transform_resNet)

test_loader_resNet = torch.utils.data.DataLoader(test_resNet, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [25]:
transform_img_aug_1 = transforms.Compose([
    transforms.RandomCrop(170),
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_img_aug_23 = transforms.Compose([
    transforms.Resize([64, 64]),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_dataset_ia_1 = ImageFolder(root='./dataset/test', transform=transform_img_aug_1)
test_dataset_ia_23 = ImageFolder(root='./dataset/test', transform=transform_img_aug_23)
test_loader_ia_1 = torch.utils.data.DataLoader(test_dataset_ia_1, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)
test_loader_ia_23 = torch.utils.data.DataLoader(test_dataset_ia_23, batch_size = BATCH_SIZE, shuffle=True, num_workers=4)

In [6]:
baseline=torch.load('./baseline/baseline.pt')
baseline.eval()
test_loss, test_accuracy = evaluate(baseline, test_loader_base)

print('baseline test accuracy: ', test_accuracy)

baseline test accuracy:  74.53333333333333


In [9]:
resnet50 = torch.load('./transfer/resnet50.pt')
resnet50.eval()
test_loss, test_accuracy = evaluate(resnet50, test_loader_resNet)

print('resNet test accuracy: ', test_accuracy)

resNet test accuracy:  93.15555555555555


In [12]:
baseline_pe=torch.load('./baseline_pumped_epoch/baseline_pumped_epoch.pt')
baseline_pe.eval()
test_loss, test_accuracy = evaluate(baseline_pe, test_loader_base)

print('baseline with more epochs test accuracy: ', test_accuracy)

baseline with more epochs test accuracy:  76.71111111111111


In [26]:
baseline_im1=torch.load('./baseline_img_aug_attempt1/baseline.pt')
baseline_im1.eval()
test_loss, test_accuracy = evaluate(baseline_im1, test_loader_ia_1)

print('img aug_v1 test accuracy: ', test_accuracy)

img aug_v1 test accuracy:  56.93333333333333


In [27]:
baseline_im2=torch.load('./baseline_img_aug_attempt2/baseline.pt')
baseline_im2.eval()
test_loss, test_accuracy = evaluate(baseline_im2, test_loader_ia_23)

print('img aug_v2 accuracy: ', test_accuracy)

img aug_v2 accuracy:  75.82222222222222


In [28]:
baseline_im3=torch.load('./baseline_img_aug_attempt3/baseline.pt')
baseline_im3.eval()
test_loss, test_accuracy = evaluate(baseline_im3, test_loader_ia_23)

print('img aug_v3 accuracy: ', test_accuracy)

img aug_v3 accuracy:  72.62222222222222
